## 1. Install and Import Libraries

In [1]:
# Install the necessary libraries
!pip install -q sentence-transformers faiss-cpu groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import the required libraries
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from groq import Groq

import os

## 2. Create the Knowledge Base

In [3]:
knowledge_base_dir = "/content/drive/MyDrive/Jupiter_Files" # The folder where your .txt files are stored
knowledge_base = [] # An empty list to hold the content of the files

for filename in os.listdir(knowledge_base_dir):
    if filename.endswith(".txt"):
        filepath = os.path.join(knowledge_base_dir, filename)
        with open(filepath, 'r', encoding='utf-8') as f:
            knowledge_base.append(f.read())

# Print the knowledge base to verify it loaded correctly
# print(knowledge_base)

## 3. Create Embeddings and the Search Index

In [5]:
# Load a pre-trained sentence-transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert our knowledge base into embeddings
knowledge_base_embeddings = model.encode(knowledge_base, convert_to_tensor=True)

# Create a FAISS index for our embeddings
index = faiss.IndexFlatL2(knowledge_base_embeddings.shape[1])

# Add our knowledge base embeddings to the index
index.add(knowledge_base_embeddings.cpu().numpy())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## 4. Define the Search Function

In [6]:
def search(query, k=1):
    """
    Takes a user's query and returns the most relevant articles.
    """
    # Convert the user's query into an embedding
    query_embedding = model.encode([query], convert_to_tensor=True)

    # Search the FAISS index for the most similar embeddings
    distances, indices = index.search(query_embedding.cpu().numpy(), k)

    # Return the most relevant articles
    return [knowledge_base[i] for i in indices[0]]

## 5. Set Up the LLM

In [7]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

# IMPORTANT: Paste your Groq API key here.
# API_KEY = "GROQ_API_KEY"

# Initialize the Groq client
client = Groq(api_key=GROQ_API_KEY)

def get_completion(prompt):
    """
    Sends a prompt to the Groq API and returns the model's response.
    """
    try:
        response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Sorry, there was an error connecting to the service."

## 6. Define the RAG System

In [13]:
def rag_system(query):
    """
    Combines the search and LLM components to create our RAG system.
    """
    # 1. Retrieve relevant articles
    relevant_articles = search(query)

    # 2. Create a prompt for the LLM
    prompt = f"""
    **Instructions:**
    - You are a helpful assistant that answers questions based ONLY on the context provided below.
    - Do NOT use any external or prior knowledge.
    - If the answer is not found in the context, you must explicitly state: "I cannot answer that question based on the provided documents."
    - Answer in bullet points if answer is long.


    Context:
    {' '.join(relevant_articles)}

    Question:
    {query}

    Answer:
    """

    # 3. Generate an answer
    return get_completion(prompt)

## 7. Ask a Question

In [14]:
# This block creates an interactive loop that lets you chat with the RAG system.
print("RAG System is ready. Ask a question or type 'exit' to stop.")
print("-" * 50)

while True:
    user_query = input("You: ")

    if user_query.lower() == 'exit':
        print("Bot: Goodbye!")
        break

    answer = rag_system(user_query)
    print(f"Bot: {answer}\n")

RAG System is ready. Ask a question or type 'exit' to stop.
--------------------------------------------------
You: hi
Bot: I cannot answer that question based on the provided documents.

You: what do u know
Bot: Based on the provided context, here's what I know:

• Jupiter has 95 moons with confirmed orbits, the most of any planet in the Solar System.
• The four largest moons, known as the Galilean moons, are Io, Europa, Ganymede, and Callisto.
• Ganymede is the largest moon in the Solar System and is larger than the planet Mercury.
• The vast majority of Jupiter's moons are small and irregular, and are thought to be captured asteroids or fragments of larger captured bodies.
• Jupiter's moons can be divided into two main groups: the inner moons and the outer, irregular moons.
• The inner moons, which include the Galilean moons, orbit in the same direction as Jupiter's rotation and have low orbital inclinations.
• The irregular moons are much farther from Jupiter and have highly inclin